<a href="https://colab.research.google.com/github/Satyajit99p/Car-insurance-Fraud-detection/blob/main/Trial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.getcwd()

'/content'

In [7]:
os.chdir('/content/drive/My Drive')

PREPROCESSING


In [8]:
import pandas as pd
import numpy as np
import seaborn as sb

In [9]:
data=pd.read_csv('my_carclaims_processed.csv')

In [10]:
data.describe()

,week_past,is_holiday,age_price_wsum,Make,AccidentArea,Sex,MaritalStatus,Fault,VehicleCategory,RepNumber,Deductible,DriverRating,Days:Policy-Accident,Days:Policy-Claim,PastNumberOfClaims,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,FraudFound
count,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000
mean,2.719048,0.038264,6.695506,11.114858,1.896426,1.843051,1.705753,1.271743,1.677865,8.482846,407.704780,2.487840,4.972696,2.993644,1.334263,5.892211,1.972242,1.994358,1.015630,2.175433,1.242623,1.098839,2.036384,0.940139
std,8.337685,0.191840,2.703195,5.282987,0.304716,0.363765,0.480789,0.444872,0.518747,4.599798,43.952379,1.119482,0.286269,0.095078,1.023565,1.395396,0.164284,0.074906,0.124043,1.249113,0.900100,0.383484,0.782336,0.237237
min,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,300.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,5.000000,7.000000,2.000000,2.000000,1.000000,1.000000,1.000000,5.000000,400.000000,1.000000,5.000000,3.000000,0.000000,5.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,0.000000,7.000000,10.000000,2.000000,2.000000,2.000000,1.000000,2.000000,8.000000,400.000000,2.000000,5.000000,3.000000,1.000000,6.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000
75%,1.000000,0.000000,8.000000,14.000000,2.000000,2.000000,2.000000,2.000000,2.000000,12.000000,400.000000,3.000000,5.000000,3.000000,2.000000,7.000000,2.000000,2.000000,1.000000,4.000000,1.000000,1.000000,3.000000,1.000000
max,51.000000,1.000000,14.000000,19.000000,2.000000,2.000000,4.000000,2.000000,3.000000,16.000000,700.000000,4.000000,5.000000,3.000000,3.000000,9.000000,2.000000,2.000000,2.000000,4.000000,5.000000,5.000000,3.000000,1.000000


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15419 entries, 0 to 15418
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   week_past             15419 non-null  int64
 1   is_holiday            15419 non-null  int64
 2   age_price_wsum        15419 non-null  int64
 3   Make                  15419 non-null  int64
 4   AccidentArea          15419 non-null  int64
 5   Sex                   15419 non-null  int64
 6   MaritalStatus         15419 non-null  int64
 7   Fault                 15419 non-null  int64
 8   VehicleCategory       15419 non-null  int64
 9   RepNumber             15419 non-null  int64
 10  Deductible            15419 non-null  int64
 11  DriverRating          15419 non-null  int64
 12  Days:Policy-Accident  15419 non-null  int64
 13  Days:Policy-Claim     15419 non-null  int64
 14  PastNumberOfClaims    15419 non-null  int64
 15  AgeOfPolicyHolder     15419 non-null  int64
 16  Poli

In [9]:
data.head(10)

,week_past,is_holiday,age_price_wsum,Make,AccidentArea,Sex,MaritalStatus,Fault,VehicleCategory,RepNumber,Deductible,DriverRating,Days:Policy-Accident,Days:Policy-Claim,PastNumberOfClaims,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,FraudFound
0,51,0,9,7,2,1,1,1,1,12,300,1,5,3,0,4,2,2,1,1,3,3,3,1
1,1,0,12,7,2,2,1,1,1,15,400,4,5,3,0,5,1,2,1,1,1,1,2,1
2,1,0,1,7,2,2,2,1,1,7,400,3,5,3,1,7,2,2,1,1,1,1,2,1
3,4,0,10,18,1,2,2,2,1,4,400,2,5,3,1,8,1,2,1,4,1,1,3,1
4,1,0,5,7,2,1,1,2,1,3,400,1,5,3,0,5,2,2,1,1,1,1,2,1
5,1,0,1,7,2,2,1,2,1,12,400,3,5,3,0,3,2,2,1,3,1,1,2,1
6,2,0,1,7,2,2,2,2,1,14,400,1,5,3,1,6,2,2,1,2,1,1,2,1
7,32,0,1,7,2,2,1,1,1,1,400,4,5,3,1,1,2,2,1,1,1,1,2,1
8,1,0,12,7,2,2,1,1,1,7,400,4,5,3,0,5,2,1,1,3,1,1,2,1
9,0,0,14,6,2,2,2,1,3,7,400,1,5,3,2,6,2,2,1,3,1,1,1,1


In [10]:
data.shape

(15419, 24)

In [11]:
df=data
type(df)

pandas.core.frame.DataFrame

In [15]:
df.shape

(15419, 24)

In [16]:
df.head(10)

,week_past,is_holiday,age_price_wsum,Make,AccidentArea,Sex,MaritalStatus,Fault,VehicleCategory,RepNumber,Deductible,DriverRating,Days:Policy-Accident,Days:Policy-Claim,PastNumberOfClaims,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,FraudFound
0,51,0,9,7,2,1,1,1,1,12,300,1,5,3,0,4,2,2,1,1,3,3,3,1
1,1,0,12,7,2,2,1,1,1,15,400,4,5,3,0,5,1,2,1,1,1,1,2,1
2,1,0,1,7,2,2,2,1,1,7,400,3,5,3,1,7,2,2,1,1,1,1,2,1
3,4,0,10,18,1,2,2,2,1,4,400,2,5,3,1,8,1,2,1,4,1,1,3,1
4,1,0,5,7,2,1,1,2,1,3,400,1,5,3,0,5,2,2,1,1,1,1,2,1
5,1,0,1,7,2,2,1,2,1,12,400,3,5,3,0,3,2,2,1,3,1,1,2,1
6,2,0,1,7,2,2,2,2,1,14,400,1,5,3,1,6,2,2,1,2,1,1,2,1
7,32,0,1,7,2,2,1,1,1,1,400,4,5,3,1,1,2,2,1,1,1,1,2,1
8,1,0,12,7,2,2,1,1,1,7,400,4,5,3,0,5,2,1,1,3,1,1,2,1
9,0,0,14,6,2,2,2,1,3,7,400,1,5,3,2,6,2,2,1,3,1,1,1,1


In [ ]:
# Data is preprocessed
# As there is a classification issue addressed, outliers are not present.
# There is no missing values and all rows contain polished data.
# 'Record_ID','Auction_ID','Bidder_ID' are non-influential in determing the desired classification.so they are omitted.

MODEL CREATION AND TRAINING

In [12]:
import tensorflow as tf
import keras
from keras.callbacks import *

In [13]:
features=df.iloc[:,:23]
labels=df.iloc[:,-1]

print(features.shape)
print(labels.shape)

(15419, 23)
(15419,)


In [14]:
from sklearn.model_selection import train_test_split

x_train_full,x_test,y_train_full,y_test=train_test_split(features,labels,random_state=9)
x_train,x_valid,y_train,y_valid=train_test_split(x_train_full,y_train_full,random_state=9)

In [15]:
x_train.shape

(8673, 23)

In [26]:
y_train.shape

(8673,)

In [27]:
x_test.shape

(3855, 23)

In [28]:
y_test.shape

(3855,)

In [29]:
x_valid.shape

(2891, 23)

In [30]:
y_valid.shape

(2891,)

In [16]:
from sklearn.utils import resample

features = pd.concat([x_train, y_train], axis=1)
not_fraud = features[features.FraudFound==1]
fraud = features[features.FraudFound==0]

fraud_unsampled=resample(fraud,
                         replace=True,
                         n_samples=len(not_fraud),
                         random_state=27
                         )

In [17]:
unsampled=pd.concat([not_fraud,fraud_unsampled])
unsampled.FraudFound.value_counts()

1    8169
0    8169
Name: FraudFound, dtype: int64

In [18]:
y_train=unsampled.FraudFound
x_train=unsampled.drop('FraudFound',axis=1)

In [19]:
print(type(not_fraud))
print(type(fraud_unsampled))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [20]:
print(x_train.shape)
print(y_train.shape)

(16338, 23)
(16338,)


In [ ]:
# there is consistent division of the dataset into the training testing and validation biparts
# 3555+1581+1185=6321

In [21]:
#np.random.seed(42)
#tf.random.set_seed(42)

**SEQUENTIAL MODEL**

In [29]:
model=keras.models.Sequential([
                               keras.layers.Dense(60,kernel_initializer='uniform',activation='relu',input_shape=[23]),
                               keras.layers.Dense(60,kernel_initializer='uniform',activation='relu'),
                               keras.layers.Dense(1,activation='sigmoid')
])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 60)                1440      
_________________________________________________________________
dense_4 (Dense)              (None, 60)                3660      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 61        
Total params: 5,161
Trainable params: 5,161
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [32]:
checkpoint=keras.callbacks.ModelCheckpoint('best-Model.h5',save_best_only=True)
early=keras.callbacks.EarlyStopping(patience=20,restore_best_weights=True)

In [33]:
model.fit(x_train,y_train,epochs=200,batch_size=10,validation_data=(x_valid,y_valid),callbacks=[checkpoint,early])

Epoch 1/200
1634/1634 [==============================] - 4s 2ms/step - loss: 0.2382 - accuracy: 0.5767 - val_loss: 0.2565 - val_accuracy: 0.4971
Epoch 2/200
1634/1634 [==============================] - 4s 2ms/step - loss: 0.2034 - accuracy: 0.6856 - val_loss: 0.2463 - val_accuracy: 0.5278
Epoch 3/200
1634/1634 [==============================] - 3s 2ms/step - loss: 0.1903 - accuracy: 0.7167 - val_loss: 0.2646 - val_accuracy: 0.5209
Epoch 4/200
1634/1634 [==============================] - 4s 2ms/step - loss: 0.1872 - accuracy: 0.7242 - val_loss: 0.1081 - val_accuracy: 0.8198
Epoch 5/200
1634/1634 [==============================] - 4s 2ms/step - loss: 0.1839 - accuracy: 0.7306 - val_loss: 0.2161 - val_accuracy: 0.5773
Epoch 6/200
1634/1634 [==============================] - 4s 2ms/step - loss: 0.1806 - accuracy: 0.7362 - val_loss: 0.1506 - val_accuracy: 0.6987
Epoch 7/200
1634/1634 [==============================] - 3s 2ms/step - loss: 0.1814 - accuracy: 0.7353 - val_loss: 0.2580 - val_ac

In [34]:
trial=keras.models.load_model('best-Model.h5')

In [35]:
test=model.evaluate(x_test,y_test)

121/121 [==============================] - 0s 1ms/step - loss: 0.1065 - accuracy: 0.8158


In [36]:
y_pred=model.predict(x_test)

In [37]:
y_pred=np.where(y_pred>=0.5,1,0)

In [38]:
y_pred=y_pred.flatten()

In [39]:
y_pred

array([1, 1, 1, ..., 0, 1, 1])

In [40]:
y_test.values

array([1, 1, 1, ..., 1, 1, 1])

In [41]:
from sklearn.metrics import confusion_matrix


confusion_matrix(y_test,y_pred)

array([[  95,  129],
       [ 581, 3050]])

In [42]:
keras.backend.clear_session()

In [43]:
del model

FUNCTIONAL MODEL

In [ ]:
input1=keras.layers.Input(shape=x_train.shape[1:])
hidden1=keras.layers.Dense(60,kernel_initializer='uniform',activation='relu')(input1)
hidden2=keras.layers.Dense(60,kernel_initializer='uniform',activation='relu')(hidden1)
concat=keras.layers.concatenate([input1,hidden2])
output=keras.layers.Dense(1,activation='sigmoid')(concat)
model=keras.models.Model(inputs=[input1],outputs=[output])

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 60)           600         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 60)           3660        dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 69)           0           input_1[0][0]                    
                                                                 dense_1[0][0]         

In [ ]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
checkpoint=keras.callbacks.ModelCheckpoint('best-Model.h5',save_best_only=True)
early=keras.callbacks.EarlyStopping(patience=20,restore_best_weights=True)

In [ ]:
model.fit(x_train,y_train,epochs=200,batch_size=10,validation_data=(x_valid,y_valid),callbacks=[checkpoint,early])

Epoch 1/200
356/356 [==============================] - 1s 3ms/step - loss: 0.0860 - accuracy: 0.8762 - val_loss: 0.0167 - val_accuracy: 0.9671
Epoch 2/200
356/356 [==============================] - 1s 3ms/step - loss: 0.0171 - accuracy: 0.9769 - val_loss: 0.0149 - val_accuracy: 0.9814
Epoch 3/200
356/356 [==============================] - 1s 3ms/step - loss: 0.0176 - accuracy: 0.9744 - val_loss: 0.0133 - val_accuracy: 0.9806
Epoch 4/200
356/356 [==============================] - 1s 3ms/step - loss: 0.0157 - accuracy: 0.9755 - val_loss: 0.0176 - val_accuracy: 0.9764
Epoch 5/200
356/356 [==============================] - 1s 3ms/step - loss: 0.0159 - accuracy: 0.9769 - val_loss: 0.0162 - val_accuracy: 0.9781
Epoch 6/200
356/356 [==============================] - 1s 3ms/step - loss: 0.0153 - accuracy: 0.9786 - val_loss: 0.0142 - val_accuracy: 0.9764
Epoch 7/200
356/356 [==============================] - 1s 3ms/step - loss: 0.0141 - accuracy: 0.9795 - val_loss: 0.0160 - val_accuracy: 0.9755

In [ ]:
trial=keras.models.load_model('best-Model.h5')

In [ ]:
test=model.evaluate(x_test,y_test)

50/50 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 0.9956


In [ ]:
keras.backend.clear_session()

In [ ]:
del model